# 工具调试 

In [1]:
# 必要包导入

from IPython.display import display, Markdown, Latex, JSON, Image, YouTubeVideo, Markdown, HTML, SVG, JSON, Pretty
from rich import print as rprint
from rich import print_json

from dotenv import load_dotenv, find_dotenv
import os 

_ = load_dotenv(find_dotenv()) # read local.env file

import nest_asyncio 
nest_asyncio.apply()

In [2]:
# change pwd 
import sys 
from pathlib import Path
cwd_dir = Path(os.getcwd())
root_dir = cwd_dir.parent
sys.path.append(str(root_dir))
print(root_dir)

/Users/guchen/repo/CyanoManus


### `wikipedia` 包

## Google Search

## PDFDEAL (doc2x SDK)

In [ ]:
from pdfdeal import Doc2X

client = Doc2X(
    apikey=os.getenv("DOC2X_API_KEY"),
    debug=True
)


success, failed, flag = client.pdf2file(
    pdf_file="toolrl.pdf",
    output_path="./output",
    output_format="docx",
)

print(success)
print(failed)
print(flag)

## Wolframe Alpha

In [ ]:
#query = "countries with the largest child population"
#query = "integrate x^2 sin^3 x dx"
query = "boiling point of mercury"

## arxiv 

https://pypi.org/project/arxiv/

In [3]:
import arxiv 
from arxiv import ArxivError, UnexpectedEmptyPageError, HTTPError 

client = arxiv.Client(
    page_size=100, # < 2000  
    delay_seconds=3.0,
    num_retries=5
)

In [9]:

search = arxiv.Search(
    query = "jupiter",  # This should be unencoded. Use `au:del_maestro AND ti:checkerboard`, not`au:del_maestro+AND+ti:checkerboard`.
    id_list=None,
    max_results = 10,
    sort_by = arxiv.SortCriterion.Relevance, # : SortCriterion  Option[.LastUpdatedDate, .Relevance, .SubmittedDate]
    sort_order = arxiv.SortOrder.Descending  # Optional: [.Ascending, .Descending]
)

try: 
    results = client.results(search)
except Exception as e:
    print(f"Exception: {e}")


In [13]:
for item in results: 
    print(item)

In [ ]:
for item in client.results(
    arxiv.Search(
        query="deepseek",
        max_results=3,
        sort_by=arxiv.SortCriterion.LastUpdatedDate,
        sort_order=arxiv.SortOrder.Descending
        )
    ):
    
    print("-"*50)
    print(f"entry_id: {item.entry_id}") # A url of the form `https://arxiv.org/abs/{id}
    print(f"updated: {item.updated}") # : class 'datetime.datetime , When the result was last updated.
    print(f"published: {item.published}") # : class 'datetime.datetime, When the result was originally published.
    print(f"title: {item.title}") # The title of the result.
    
    # 将作者名字拼接成一个字符串，跳过空值
    author_names = ", ".join([author.name for author in item.authors if author.name is not None])
    print(f"authors: {author_names}")
    
    
    print(f"summary: {item.summary}") # The result abstract.
    print(f"comment: {item.comment}") # The authors' comment if present
    print(f"journal_ref: {item.journal_ref}") # A journal reference if present.
    print(f"doi: {item.doi}") # A URL for the resolved DOI to an external resource if present.
    

    print(f"primary_category: {item.primary_category}") # :str, The result's primary arXiv category. See [arXiv: Category Taxonomy](https://arxiv.org/category_taxonomy).
    print(f"categories: {item.categories}") # :list[str], All of the result's categories. See [arXiv: Category Taxonomy](https://arxiv.org/category_taxonomy)
    
    print(f"links: {item.links}") # : list[arxiv.Result.Link], Up to three URLs associated with this result.
    
    
    print(f"pdf_url: {item.pdf_url}") # The URL of a PDF version of this result if present among links
    #print(f"_raw: {item._raw}") # The raw feedparser result object if this Result was constructed with Result._from_feed_entry.
    print("-"*50)
    
    item.download_pdf()
    
    
    

--------------------------------------------------
entry_id: http://arxiv.org/abs/2503.16419v3
updated: 2025-04-23 17:46:54+00:00
published: 2025-03-20 17:59:38+00:00
title: Stop Overthinking: A Survey on Efficient Reasoning for Large Language Models
authors: Yang Sui, Yu-Neng Chuang, Guanchu Wang, Jiamu Zhang, Tianyi Zhang, Jiayi Yuan, Hongyi Liu, Andrew Wen, Shaochen Zhong, Hanjie Chen, Xia Hu
summary: Large Language Models (LLMs) have demonstrated remarkable capabilities in
complex tasks. Recent advancements in Large Reasoning Models (LRMs), such as
OpenAI o1 and DeepSeek-R1, have further improved performance in System-2
reasoning domains like mathematics and programming by harnessing supervised
fine-tuning (SFT) and reinforcement learning (RL) techniques to enhance the
Chain-of-Thought (CoT) reasoning. However, while longer CoT reasoning sequences
improve performance, they also introduce significant computational overhead due
to verbose and redundant outputs, known as the "overthin

In [15]:
# search specific paper with id 

paper = next(arxiv.Client().results(arxiv.Search(id_list=["2503.16419v3"])))

In [ ]:
paper = next(arxiv.Client().results(arxiv.Search(id_list=["1605.08386v1"])))
# Download the PDF to the PWD with a default filename.
paper.download_pdf()
# Download the PDF to the PWD with a custom filename.
paper.download_pdf(filename="downloaded-paper.pdf")
# Download the PDF to a specified directory with a custom filename.
paper.download_pdf(dirpath="./mydir", filename="downloaded-paper.pdf")

'./2503.16419v3.Stop_Overthinking__A_Survey_on_Efficient_Reasoning_for_Large_Language_Models.pdf'

In [ ]:
# Search for the paper with ID "1605.08386v1"
search_by_id = arxiv.Search(id_list=["1605.08386v1"])
# Reuse client to fetch the paper, then print its title.
first_result = next(client.results(search))
print(first_result.title)

In [ ]:
# To download a PDF of the paper with ID "1605.08386v1," run a Search and then use Result.download_pdf():

paper = next(arxiv.Client().results(arxiv.Search(id_list=["1605.08386v1"])))
# Download the PDF to the PWD with a default filename.
paper.download_pdf()
# Download the PDF to the PWD with a custom filename.
paper.download_pdf(filename="downloaded-paper.pdf")
# Download the PDF to a specified directory with a custom filename.
paper.download_pdf(dirpath="./mydir", filename="downloaded-paper.pdf")